In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
df_boston = pd.read_csv('../data/raw/boston_weather_data.csv')

In [5]:
df_boston

,time,tavg,tmin,tmax,prcp,wdir,wspd,pres
0,2013-03-01,3.2,1.1,5.0,0.0,342.0,15.1,1002.7
1,2013-03-02,3.1,1.1,5.6,0.0,307.0,14.4,1004.2
2,2013-03-03,2.6,0.6,6.1,0.0,NaN,14.4,1002.6
3,2013-03-04,1.8,-0.6,5.0,0.0,303.0,28.1,1003.0
4,2013-03-05,2.9,0.0,6.1,0.0,NaN,16.6,1013.5
...,...,...,...,...,...,...,...,...
3648,2023-02-25,-7.9,-10.6,-6.7,0.3,313.0,11.9,1029.5
3649,2023-02-26,-4.2,-7.8,3.9,0.0,268.0,11.9,1012.6
3650,2023-02-27,-1.7,-2.8,-0.6,0.0,46.0,18.7,1014.9
3651,2023-02-28,0.6,-1.7,2.2,7.1,86.0,23.8,1010.7


clean the column names
see if we can combine this with golf data.
we only care about tavg for temp and we dont need wdir or pres
rename time to date and that matters a lot!! keep the same format with the golf rounds. that dataset needs the most cleaning



In [6]:
df_golf = pd.read_csv('../data/raw/golf.csv')

In [7]:
df_golf

,Year,Name,Course,Yards,Par,Open.Position,Integer.Position,Position.Grouping,Open.R1,Open.R2,...,DistanceOfPuttAVG,SGAroundTheGreenAVG,SGOffTheTeeAVG,SGTeeToGreenAVG,SGTeeToGreenAVGOTT,SGTeeToGreenAVGAPR,SGTeeToGreenAVGARG,SGTeeToGreenAVGPUTT,SGTotalAVG,BirdOrBetterFromFairway.
0,2021,Collin Morikawa,Royal St Georges,7189,70,1,1,1,67,64,...,"70' 7""",0.077,0.281,1.528,0.281,1.170,0.077,-0.457,1.071,26.16
1,2021,Jordan Spieth,Royal St Georges,7189,70,2,2,1,65,67,...,"77' 2""",0.407,-0.091,0.680,-0.091,0.365,0.407,0.389,1.069,28.21
2,2021,Dylan Frittelli,Royal St Georges,7189,70,5,5,1,66,67,...,"66' 4""",0.312,0.029,0.009,0.029,-0.333,0.312,-0.470,-0.461,21.81
3,2021,Kevin Kisner,Royal St Georges,7189,70,73,73,8,70,69,...,"78' 7""",0.085,-0.215,-0.396,-0.215,-0.266,0.085,0.392,-0.003,24.03
4,2021,Phil Mickelson,Royal St Georges,7189,70,CUT,90,10,80,72,...,"74' 9""",-0.107,-0.161,-0.410,-0.161,-0.142,-0.107,-0.032,-0.442,22.61
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1974,1993,Dan Forsman,Royal St Georges,6860,70,T73,73,8,71,70,...,0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.00
1975,1993,Wayne Grady,Royal St Georges,6860,70,T9,9,1,74,68,...,0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.00
1976,1993,Scott Simpson,Royal St Georges,6860,70,T9,9,1,68,70,...,0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.00
1977,1993,Fred Couples,Royal St Georges,6860,70,T9,9,1,68,66,...,0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.00


Convert Name to a number to use as a player ID (could keep name as well and just assign an ID to each)

extract all 4 rounds scores per player so we have a table of player, round num, round score, date(needs to be generated somewhat randomly), and handicap ( can calc this on our own and never update it in the db or have it built into site to update at request). We need to have it be incremental for overall rounds by the player and that needs to be ordered by date when calculating.

The year can stay the same and we can just use the same years from the weather data from boston

our course par is 72 and it is located in boston

make sure the months assigned are between april and october.

We can also use this data set for and overall player table just for fun. like scoring averages and par3,4,5 averages and swing stats